In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
customer_rep = Path.cwd() / 'data' / 'processed' / 'customer_rep_data.xlsx'
invoice_data = Path.cwd() / 'data' / 'raw' / 'invoice_data.xlsx'
report_file = Path.cwd() / 'reports' / 'commission_report.xlsx'

In [3]:
raw_df = pd.read_excel(customer_rep, usecols='A,B,F,E,H,I,J')

In [4]:
raw_df.head()

,company_name,channel,state,account_num,Region,first_name,last_name
0,Universal Technology Vision,retail,VA,AH5590,SOUTH,Mickey,Tyner
1,Contract Electronics Industries,retail,VA,GG0303,SOUTH,Mickey,Tyner
2,Star Interactive,retail,VA,UM2244,SOUTH,Mickey,Tyner
3,Vision People Solutions,retail,VA,WL5283,SOUTH,Mickey,Tyner
4,East Design Hill,retail,KS,OL0453,MIDWEST,Mona,Sutton


In [5]:
# Just get the columns we need
customers = raw_df.copy()

In [6]:
customers.head()

,company_name,channel,state,account_num,Region,first_name,last_name
0,Universal Technology Vision,retail,VA,AH5590,SOUTH,Mickey,Tyner
1,Contract Electronics Industries,retail,VA,GG0303,SOUTH,Mickey,Tyner
2,Star Interactive,retail,VA,UM2244,SOUTH,Mickey,Tyner
3,Vision People Solutions,retail,VA,WL5283,SOUTH,Mickey,Tyner
4,East Design Hill,retail,KS,OL0453,MIDWEST,Mona,Sutton


In [7]:
customers.shape

(50, 7)

In [8]:
invoice = pd.read_excel(invoice_data)

In [9]:
invoice.head()

,cust_num,sku,qty,list_price,discount_rate,invoice_price,invoice_num,invoice_date_time,invoice_total
0,LA6029,SW200,4,20000,0.24,15200.0,98105,2019-12-13 14:11:43.828,60800.0
1,EB0265,PS501,4,30000,0.10,27000.0,58436,2019-06-05 23:12:47.344,108000.0
2,EE4079,SW500,1,20000,0.36,12800.0,85825,2019-09-12 03:23:24.309,12800.0
3,YR6861,ACC5144,4,400,0.12,352.0,46422,2019-10-10 15:02:54.590,1408.0
4,WL5283,SW200,1,20000,0.17,16600.0,34838,2019-08-03 11:32:29.245,16600.0


In [10]:
invoice.shape

(2000, 9)

In [11]:
invoice['invoice_total'].sum()

126493662.0

In [12]:
# Merge in Customer name, channel and region
# Different column names for left and right
sales_detail = pd.merge(invoice, customers, left_on='cust_num', right_on='account_num')

In [13]:
sales_detail.shape

(2000, 16)

In [14]:
sales_detail.head()

,cust_num,sku,qty,list_price,discount_rate,invoice_price,invoice_num,invoice_date_time,invoice_total,company_name,channel,state,account_num,Region,first_name,last_name
0,LA6029,SW200,4,20000,0.24,15200.0,98105,2019-12-13 14:11:43.828,60800.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm
1,LA6029,ACC5144,3,400,0.19,324.0,77479,2019-01-17 03:26:03.748,972.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm
2,LA6029,SW121,1,20000,0.16,16800.0,10471,2019-05-26 00:27:56.565,16800.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm
3,LA6029,SW200,3,20000,0.11,17800.0,12004,2019-01-14 14:19:24.701,53400.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm
4,LA6029,SW121,2,20000,0.10,18000.0,14940,2019-06-04 07:18:48.335,36000.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm


In [15]:
sales_detail['invoice_total'].sum()

126493662.0

In [16]:
commission_rate = 0.0079

In [17]:
sales_detail['commission'] = commission_rate * sales_detail['invoice_total']

In [18]:
sales_detail['commission'].sum()

999299.9298

In [19]:
sales_detail.groupby(['sku']).agg({'commission': 'sum'}).sort_values(by=['commission'], ascending=False)

,commission
sku,
SW500,236096.2400
SW200,206667.1600
SW121,204265.5600
PS501,147018.2100
PS403,127432.5300
SPA1,25867.3650
SPB1,25580.9900
SPBC2,22069.4400
ACC5144,1200.1048


In [20]:
sales_detail.groupby(['last_name']).agg({'commission': 'sum'})

,commission
last_name,
Malcolm,101987.1988
Muniz,101517.8756
Sutton,441927.5326
Tyner,353867.3228


In [21]:
sales_detail.groupby(['last_name', 'channel']).agg({'commission': 'sum'})

commission
last_name channel              
Malcolm   reseller   62041.6704
          retail     39945.5284
Muniz     reseller   49158.2556
          retail     52359.6200
Sutton    reseller  356020.0728
          retail     85907.4598
Tyner     partner    55320.6348
          reseller  107175.3816
          retail    191371.3064

In [22]:
pd.pivot_table(data=sales_detail, index=['last_name'], columns=['channel'], values='commission', aggfunc='sum').round(0)

channel,partner,reseller,retail
last_name,,,
Malcolm,NaN,62042.0,39946.0
Muniz,NaN,49158.0,52360.0
Sutton,NaN,356020.0,85907.0
Tyner,55321.0,107175.0,191371.0


In [23]:
# What happens if we adjust the partner & reseller commision rates down?
# Sales team does not do as much work for these channels
partner_rate = 0.001
reseller_rate = 0.0026

In [24]:
# Add the commission rate as a column with default of the base
sales_detail['commission_rate'] = commission_rate

In [25]:
partners = (sales_detail['channel'] == 'partner')
resellers = (sales_detail['channel'] == 'reseller')
retail = (sales_detail['channel'] == 'retail')

In [26]:
partners

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Name: channel, Length: 2000, dtype: bool

In [27]:
sales_detail.loc[partners, 'commission_rate'] = partner_rate
sales_detail.loc[resellers, 'commission_rate'] = reseller_rate

In [28]:
sales_detail.head()

,cust_num,sku,qty,list_price,discount_rate,invoice_price,invoice_num,invoice_date_time,invoice_total,company_name,channel,state,account_num,Region,first_name,last_name,commission,commission_rate
0,LA6029,SW200,4,20000,0.24,15200.0,98105,2019-12-13 14:11:43.828,60800.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,480.3200,0.0079
1,LA6029,ACC5144,3,400,0.19,324.0,77479,2019-01-17 03:26:03.748,972.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,7.6788,0.0079
2,LA6029,SW121,1,20000,0.16,16800.0,10471,2019-05-26 00:27:56.565,16800.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,132.7200,0.0079
3,LA6029,SW200,3,20000,0.11,17800.0,12004,2019-01-14 14:19:24.701,53400.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,421.8600,0.0079
4,LA6029,SW121,2,20000,0.10,18000.0,14940,2019-06-04 07:18:48.335,36000.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,284.4000,0.0079


In [29]:
sales_detail['commission_rate'].value_counts()

0.0079    1503
0.0026     422
0.0010      75
Name: commission_rate, dtype: int64

In [30]:
# Unlike excel - you need to tell it to calculate
sales_detail['commission'] = sales_detail['commission_rate'] * sales_detail['invoice_total']

In [31]:
sales_detail.head()

,cust_num,sku,qty,list_price,discount_rate,invoice_price,invoice_num,invoice_date_time,invoice_total,company_name,channel,state,account_num,Region,first_name,last_name,commission,commission_rate
0,LA6029,SW200,4,20000,0.24,15200.0,98105,2019-12-13 14:11:43.828,60800.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,480.3200,0.0079
1,LA6029,ACC5144,3,400,0.19,324.0,77479,2019-01-17 03:26:03.748,972.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,7.6788,0.0079
2,LA6029,SW121,1,20000,0.16,16800.0,10471,2019-05-26 00:27:56.565,16800.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,132.7200,0.0079
3,LA6029,SW200,3,20000,0.11,17800.0,12004,2019-01-14 14:19:24.701,53400.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,421.8600,0.0079
4,LA6029,SW121,2,20000,0.10,18000.0,14940,2019-06-04 07:18:48.335,36000.0,Bell Frontier Resource,retail,CA,LA6029,WEST,Leonard,Malcolm,284.4000,0.0079


In [32]:
sales_detail['commission'].sum()

565628.0442000001

In [33]:
pd.pivot_table(data=sales_detail,
               index=['last_name'],
               columns=['channel'],
               values='commission',
               margins=True,
               aggfunc='sum').round(0)

channel,partner,reseller,retail,All
last_name,,,,
Malcolm,NaN,20419.0,39946.0,60364.0
Muniz,NaN,16179.0,52360.0,68538.0
Sutton,NaN,117171.0,85907.0,203079.0
Tyner,7003.0,35273.0,191371.0,233647.0
All,7003.0,189042.0,369584.0,565628.0


In [34]:
# Shifted things around. Maybe bump up to a different rate?
# Try a loop
retail_rates = [0.01, 0.012, 0.014, 0.016, 0.017]
for rate in retail_rates:
    sales_detail.loc[retail, 'commission_rate'] = rate
    sales_detail['commission'] = sales_detail['commission_rate'] * sales_detail['invoice_total']
    print(f"Total Commmissions {sales_detail['commission'].sum():,.0f} retail rate= {rate}")
    

Total Commmissions 663,872 retail rate= 0.01
Total Commmissions 757,437 retail rate= 0.012
Total Commmissions 851,003 retail rate= 0.014
Total Commmissions 944,569 retail rate= 0.016
Total Commmissions 991,351 retail rate= 0.017


In [35]:
pd.pivot_table(data=sales_detail,
               index=['last_name'],
               columns=['channel'],
               values='commission',
               margins=True,
               aggfunc='sum').round(0)

channel,partner,reseller,retail,All
last_name,,,,
Malcolm,NaN,20419.0,85959.0,106378.0
Muniz,NaN,16179.0,112673.0,128851.0
Sutton,NaN,117171.0,184864.0,302035.0
Tyner,7003.0,35273.0,411812.0,454087.0
All,7003.0,189042.0,795307.0,991351.0


In [36]:
sales_detail.groupby(['last_name', 'state']).agg({'commission': 'sum'})

commission
last_name state            
Malcolm   AZ     20290.5200
          CA     49381.5320
          ID     20418.7776
          OR     16286.6800
Muniz     CT     49809.1160
          MA     16559.4960
          ME     26025.5040
          NY     20278.4840
          PA     16178.6664
Sutton    IA     17025.3720
          IL     18092.1000
          KS     57142.8248
          MI     75877.6300
          MN     17767.0584
          MO     55340.9904
          NE     44774.5936
          OH     16014.7480
Tyner     AL     24294.4620
          DC     21002.2400
          FL     20909.0220
          GA     17344.3860
          KY     18724.0380
          LA     17050.6128
          MD     15573.5980
          MS     22661.9520
          NC     38086.1856
          OK     16505.0620
          SC     38248.0280
          TX     52774.3580
          VA     87276.2660
          WV     63636.9840

In [37]:
# Move VA and WV to the NE region
# Move Nebraska to the W

In [38]:
VA_WV = sales_detail['state'].isin(['VA', 'WV'])
NEB = sales_detail['state'].isin(['NE'])
sales_detail.loc[VA_WV, ['Region', 'last_name', 'first_name']] = ('NORTHEAST', 'Muniz', 'Shannon')
sales_detail.loc[NEB, ['Region', 'last_name', 'first_name']] = ('WEST', 'Malcolm', 'Leonard')

In [39]:
commission_by_channel = pd.pivot_table(data=sales_detail,
                                       index=['last_name'],
                                       columns=['channel'],
                                       values='commission',
                                       margins=True,
                                       fill_value=0,
                                       aggfunc='sum').round(0)

In [40]:
commission_by_channel

channel,partner,reseller,retail,All
last_name,,,,
Malcolm,0.0,44135.0,107017.0,151152.0
Muniz,0.0,16179.0,263586.0,279765.0
Sutton,0.0,93455.0,163806.0,257261.0
Tyner,7003.0,35273.0,260898.0,303174.0
All,7003.0,189042.0,795307.0,991351.0


In [41]:
commission_rates = sales_detail.groupby(['channel']).agg({'commission_rate': 'mean'})
commission_rates

,commission_rate
channel,
partner,0.0010
reseller,0.0026
retail,0.0170


In [42]:
new_alignments = sales_detail.groupby(['last_name', 'state']).agg({'account_num': 'size', 'invoice_total': 'sum'})
new_alignments

account_num  invoice_total
last_name state                            
Malcolm   AZ              43      1193560.0
          CA              92      2904796.0
          ID              40      7853376.0
          NE              84     10360328.0
          OR              25       958040.0
Muniz     CT             102      2929948.0
          MA              32       974088.0
          ME              44      1530912.0
          NY              40      1192852.0
          PA              34      6222564.0
          VA             146      5133898.0
          WV             121      3743352.0
Sutton    IA              41      6548220.0
          IL              48      6958500.0
          KS             106      9118012.0
          MI             145      4463390.0
          MN              41      6833484.0
          MO             119     10716216.0
          OH              33       942044.0
Tyner     AL              38      1429086.0
          DC              76      4828288.0
          FL              67      4227774.0
          GA              34      1020258.0
          KY              43      1101414.0
          LA              38      6557928.0
          MD              34       916094.0
          MS              42      1333056.0
          NC              84      8177040.0
          OK              40       970886.0
          SC              80      2249884.0
          TX              88      3104374.0

In [43]:
# Write out the new rates, expected commissions and alignment

In [44]:
output = pd.ExcelWriter(report_file)

In [45]:
# Format the numbers
commission_by_channel.to_excel(output)
commission_rates.to_excel(output, startcol=6)
new_alignments.to_excel(output, startcol=9)

In [46]:
output.save()